In [225]:
import sys
sys.path.append("..")

import plotly.express as px
import plotly.io as pio
from history_parser import HistoryParser
from hand_parser import HandParser
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# Create a custom template
custom_theme = go.layout.Template(
    layout=go.Layout(
        plot_bgcolor='rgba(0, 0, 0, 0)',  # Transparent plot background
        paper_bgcolor='rgba(10, 10, 10, 1)',  # Dark paper background
        font_color='white',  # White font for visibility
        title_font_size=20,
        legend_title_font_size=16,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        ),
        xaxis=dict(
            title_font_size=16,
            showgrid=True,
            gridwidth=1,
            gridcolor='DimGray'
        ),
        yaxis=dict(
            title_font_size=16,
            showgrid=True,
            gridwidth=1,
            gridcolor='DimGray',
            autorange=True
        )
    )
)

pio.templates["custom_dark_theme"] = custom_theme
pio.templates.default = "custom_dark_theme"


In [226]:
hands = HandParser.parse_folder("../../games/3.8/hand_data")

In [227]:
cnt = {}
for h in hands:
	for player in h:
		if player not in cnt:
			cnt[player] = 0
		cnt[player] += 1

# Convert the data into a pandas DataFrame
known_df = pd.DataFrame({'Username': [player.username for player in cnt.keys()],
                   'Known Hands': [hands for hands in cnt.values()]})

known_df = known_df.sort_values('Known Hands', ascending=False)

# Plot the data using Plotly Express
fig = px.bar(known_df, x='Username', y='Known Hands', text='Known Hands',
             title="Number of Known Hands by Player")

# Update layout for clarity
fig.update_layout(
                  xaxis_tickangle=-45,
                  yaxis=dict(title='Number of Known Hands'),
                  xaxis=dict(title='Username'))

fig.show()

In [228]:
def hand_rank(hand):
    rank_map = {1: "A", 11: "J", 12: "Q", 13: "K"}
    
    rankings = ["A,A","K,K","Q,Q","A,K suited","J,J","A,Q suited","K,Q suited","A,J suited","J,K suited","10,10","A,K","10,A suited","J,Q suited","10,K suited","10,Q suited","10,J suited","9,9","A,Q","9,A suited","K,Q","8,8","9,K suited","10,9 suited","8,A suited","9,Q suited","9,J suited","A,J","5,A suited","7,7","7,A suited","J,K","4,A suited","3,A suited", "6,A suited","J,Q","6,6","8,K suited","10,8 suited","2,A suited","8,9 suited","8,J suited","10,A","8,Q suited","7,K suited","10,K","5,5","10,J","7,8 suited","10,Q","4,4","2,2","3,3","6,K suited","7,9 suited","5,K suited","6,7 suited","10,7 suited","4,K suited","3,K suited","2,K suited","7,Q suited","6,8 suited","5,6 suited","7,J suited","4,5 suited","6,Q suited","5,7 suited","6,9 suited","5,Q suited","4,6 suited","4,Q suited","3,Q suited","10,9","10,6 suited","2,Q suited","9,A","3,5 suited","5,8 suited","6,J suited","9,J","9,K","5,J suited","9,Q","3,4 suited","4,7 suited","4,J suited","3,J suited","5,9 suited","2,J suited","3,6 suited","8,A","2,5 suited","10,5 suited","4,8 suited","10,4 suited","10,3 suited","2,4 suited","10,2 suited","8,9","10,8","5,A","7,A","3,7 suited","4,A","2,3 suited","4,9 suited","3,9 suited","8,J","3,A","2,6 suited","2,9 suited","8,K","6,A","7,8","8,Q","3,8 suited","2,A","2,8 suited","7,9","2,7 suited","6,7","7,K","5,6","10,7","6,K","6,8","4,5","5,K","7,J","5,7","7,Q","4,K","3,K","2,K","6,9","4,6","6,Q","3,5","5,8","10,6","5,Q","3,4","4,Q","3,Q","2,Q","4,7","6,J","3,6","5,J","5,9","2,5","4,J","3,J","2,4","2,J","4,8","10,5","10,4","10,3","10,2","2,3","3,7","2,6","4,9","3,9","2,9","3,8","2,8","2,7"]
    
    # Convert the input hand to its string representation
    hand_str = ','.join(sorted([str(rank_map.get(card.rank, card.rank)) for card in hand], reverse=False))
    suited_str = " suited" if hand[0].suit == hand[1].suit else ""
    
    # Construct the representation used in the rankings list
    lookup_str = f"{hand_str}{suited_str}"
    
    # Find and return the ranking position
    return rankings.index(lookup_str) + 1  # +1 because list indices start at 0, but rankings start at 1

In [229]:
from structs import Player

hand_rankings: dict[Player, list[int]] = {}
player_hands = {}
for hand in hands:
	for player in hand:
		if(player.username not in hand_rankings):
			hand_rankings[player.username] = []
			player_hands[player.username] = []
		hand_rankings[player.username].append(hand_rank(hand[player]))
		player_hands[player.username].append(hand[player])


In [254]:

all_rankings = []

for player in {"yyj", "Jonathan", "az", "michael"}:
	all_rankings.extend(hand_rankings[player])

# Create a histogram with Plotly Graph Objects for finer control over bins
fig = go.Figure(data=[go.Histogram(
    x=all_rankings,
    xbins=dict(  # Controls the bins
        start=0.5,
        end=169.5,
        size=13
    ),
    marker=dict(color='royalblue')
)])

# Update layout to add custom axis titles
fig.update_layout(
    title="Distribution of Hand Rankings",
    xaxis_title="Hand Ranking",
    yaxis_title="Number of Occurrences",
    xaxis=dict(tickmode='linear', tick0=1, dtick=10),
    bargap=0.1  # Adjust the gap between bars
)

fig.show()

In [264]:
# Create a figure
fig = go.Figure()

bin_edges = np.linspace(0, 169, 52)  # 10 bins for rankings 1-100
def get_bin_counts(rankings, bins):
    """Count occurrences in each bin for the given rankings."""
    counts, _ = np.histogram(rankings, bins=bins)
    return counts

# Add a line for the binned data of each player
for player, rankings in hand_rankings.items():
    # if player not in {"yyj", "Jonathan", "az", "michael"}:
    #     continue
    bin_counts = np.cumsum(get_bin_counts(rankings, bin_edges) / len(rankings))
    # Using the midpoint of bins for x-axis values
    bin_midpoints = (bin_edges[:-1] + bin_edges[1:]) / 2
    fig.add_trace(go.Scatter(x=bin_midpoints, y=bin_counts, mode='lines', name=player))

bin_counts = np.cumsum(get_bin_counts(all_rankings, bin_edges) / len(all_rankings))
# Using the midpoint of bins for x-axis values
bin_midpoints = (bin_edges[:-1] + bin_edges[1:]) / 2
fig.add_trace(go.Scatter(x=bin_midpoints, y=bin_counts, mode='lines', name="Combined"))

# Update the layout for readability
fig.update_layout(
    title='Cumulative Probability of Getting a Ranked Hand',
    xaxis_title='Ranking of Hand',
    yaxis_title='Probability',
)

# Show the figure
fig.show()

In [262]:
import plotly.figure_factory as ff
import plotly.graph_objects as go

from structs import Card

ranks = ["A", "K", "Q", "J", "T", "9", "8", "7", "6", "5", "4", "3", "2"]

def plot_range(cards: list[tuple[Card, Card]], title="") -> None:
    def get_label(r1, r2):
        suited = False
        if(r2 > r1):
            r1, r2 = r2, r1
            suited = True
        return f'{ranks[r2]}{ranks[r1]}{"s" if suited else ("o" if r1 != r2 else "")}'

    heatmap_values = [[0 for _ in range(13)] for _ in range(13)]
    string_matrix = [[get_label(12-i,j) for j in range(13)] for i in range(13)]

    for hole in cards:
        c1, c2 = hole
        if(c1.suit != c2.suit):
            heatmap_values[c1.rank-2][(14-c2.rank)%13] += 1
        else:
            heatmap_values[c2.rank-2][(14-c1.rank)%13] += 1

    fig = ff.create_annotated_heatmap(
        z=heatmap_values,
        annotation_text=string_matrix,
        colorscale='blues'
    )

    # Adjusting the layout to attempt square cells
    # Note: These dimensions might need adjustment based on your display or specific needs
    fig.update_layout(
        width=650,  # Adjust the figure width
        height=650,  # Adjust the figure height to match the width for square cells
        autosize=False,
        xaxis=dict(constrain='domain'),  # This can help maintain proportions
        yaxis=dict(scaleanchor="x", scaleratio=1),  # This forces the y-axis to scale with the x-axis
    )

    # Additional layout adjustments (optional)
    fig.update_layout(
        title=title,
        xaxis=dict(side="bottom"),
    )

    fig.show()


In [273]:
for player in player_hands:
	plot_range(player_hands[player], title=f"{player}'s Cards")